In [1]:
from dp_policy.experiments import titlei_funding as funding, titlei_grid as test_params
from dp_policy.titlei.allocators import AbowdAllocator, SonnenbergAuthorizer
from dp_policy.titlei.utils import weighting
from dp_policy.titlei.mechanisms import Laplace, Gaussian, GroundTruth
from dp_policy.titlei.utils import *
from dp_policy.titlei.thresholders import *
from dp_policy.titlei.evaluation import *
from dp_policy.experiments import *

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import seaborn as sns



In [2]:
saipe = get_inputs(2021)
sppe = get_sppe("../data/sppe18.xlsx")
print(saipe.shape)

plt.style.use('seaborn-deep')

%matplotlib inline
%load_ext autoreload
%autoreload 2

/Users/kentarohoffman/Desktop/dp-policy-master-2


/Users/kentarohoffman/Desktop/dp-policy-master-2/dp_policy/titlei/utils.py:70: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  combined.loc[:, "State FIPS Code"] = \


[INFO] Successfully imputed 2937 new indices
-- WARNING: dropping some balances from total budget --
Puerto Rico 412450771.2306171
County balances 723748.7996060263
Part D Subpart 2 104168000.98352575
(13190, 25)


/var/folders/0s/_wb58sfj7zlbs9_f_fpks_ym0000gn/T/ipykernel_26652/2294734140.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-deep')


In [3]:
saipe.head()


Name  \
State FIPS Code District ID                                        
1               190               Alabaster City School District   
                5               Albertville City School District   
                30           Alexander City City School District   
                60                Andalusia City School District   
                90                 Anniston City School District   

                             official_basic_hold_harmless  \
State FIPS Code District ID                                 
1               190                         451569.585546   
                5                           820486.906460   
                30                          548830.474149   
                60                          255888.455695   
                90                          690724.128654   

                             official_concentration_hold_harmless  \
State FIPS Code District ID                                         
1               190                                  58092.627841   
                5                                   200836.024036   
                30                                  135670.119992   
                60                                   63255.265741   
                90                                  170746.031480   

                             official_targeted_hold_harmless  \
State FIPS Code District ID                                    
1               190                            182516.637263   
                5                              553590.326339   
                30                             392958.114409   
                60                             157617.381627   
                90                             537272.595916   

                             official_efig_hold_harmless  \
State FIPS Code District ID                                
1               190                        194698.670798   
                5                          592983.087034   
                30                         419186.018887   
                60                         168137.519722   
                90                         573132.739294   

                             official_total_hold_harmless  \
State FIPS Code District ID                                 
1               190                          8.868775e+05   
                5                            2.167896e+06   
                30                           1.496645e+06   
                60                           6.448986e+05   
                90                           1.971875e+06   

                             official_basic_alloc  \
State FIPS Code District ID                         
1               190                 404482.451481   
                5                   738438.215814   
                30                  493947.426734   
                60                  245814.743916   
                90                  662918.490317   

                             official_concentration_alloc  \
State FIPS Code District ID                                 
1               190                              0.000000   
                5                           180752.421632   
                30                          122103.107993   
                60                           62375.463078   
                90                          168215.490893   

                             official_targeted_alloc  official_efig_alloc  \
State FIPS Code District ID                                                 
1               190                    159746.293284        166408.047625   
                5                      498231.293705        533684.778331   
                30                     353662.302968        377267.416998   
                60                     144408.067790        151323.767750   
                90                     510408.966121        544476.102330   

                             ... 

In [4]:
sppe.head()

,Name,ppe,Postal Code
State FIPS Code,,,
1,Alabama,9697,AL
2,Alaska,17726,AK
4,Arizona,8296,AZ
5,Arkansas,10072,AR
6,California,12510,CA


In [5]:
saipe.columns

Index(['Name', 'official_basic_hold_harmless',
       'official_concentration_hold_harmless',
       'official_targeted_hold_harmless', 'official_efig_hold_harmless',
       'official_total_hold_harmless', 'official_basic_alloc',
       'official_concentration_alloc', 'official_targeted_alloc',
       'official_efig_alloc', 'official_total_alloc',
       'official_hold_harmless_percentage', 'official_total_formula_count',
       'official_5-17_pop.', 'official_percent_formula',
       'official_basic_eligibles', 'official_concentration_eligibles',
       'official_targeted_eligibles', 'official_weighted_counts_targeted',
       'official_weighted_counts_efig', 'State Postal Code',
       'Estimated Total Population', 'Estimated Population 5-17',
       'Estimated number of relevant children 5 to 17 years old in poverty who are related to the householder',
       'cv'],
      dtype='object')

In [14]:
official_inputs = get_inputs(2021, use_official_children=True)
allocs1 = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H_prop_laplace'})

/Users/kentarohoffman/Desktop/dp-policy-master-2


/Users/kentarohoffman/Desktop/dp-policy-master-2/dp_policy/titlei/utils.py:70: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  combined.loc[:, "State FIPS Code"] = \


[INFO] Successfully imputed 2937 new indices
-- WARNING: dropping some balances from total budget --
Puerto Rico 412450771.2306171
County balances 723748.7996060263
Part D Subpart 2 104168000.98352575
State FIPS Code  District ID
1                190             649.0
                 5               957.0
                 30              703.0
                 60              406.0
                 90             1057.0
                                 ...  
56               4260             28.0
                 6240            148.0
                 5820             12.0
                 4830            108.0
                 6090             12.0
Name: true_children_poverty, Length: 13190, dtype: float64
State FIPS Code  District ID
1                190             639.031740
                 5               959.046797
                 30              709.575386
                 60              407.103426
                 90             1066.821329
                                 

In [11]:
allocs1

Name  \
State FIPS Code District ID                                        
1               190               Alabaster City School District   
                5               Albertville City School District   
                30           Alexander City City School District   
                60                Andalusia City School District   
                90                 Anniston City School District   
...                                                          ...   
56              4260              Uinta County School District 6   
                6240           Washakie County School District 1   
                5820           Washakie County School District 2   
                4830             Weston County School District 1   
                6090             Weston County School District 7   

                             official_basic_hold_harmless  \
State FIPS Code District ID                                 
1               190                         451569.585546   
                5                           820486.906460   
                30                          548830.474149   
                60                          255888.455695   
                90                          690724.128654   
...                                                   ...   
56              4260                         40822.247350   
                6240                        223321.706091   
                5820                             0.000000   
                4830                        182499.458741   
                6090                         20411.123675   

                             official_concentration_hold_harmless  \
State FIPS Code District ID                                         
1               190                                  58092.627841   
                5                                   200836.024036   
                30                                  135670.119992   
                60                                   63255.265741   
                90                                  170746.031480   
...                                                           ...   
56              4260                                     0.000000   
                6240                                 24485.720477   
                5820                                  5762.953584   
                4830                                 93609.867864   
                6090                                     0.000000   

                             official_targeted_hold_harmless  \
State FIPS Code District ID                                    
1               190                            182516.637263   
                5                              553590.326339   
                30                             392958.114409   
                60                             157617.381627   
                90                             537272.595916   
...                                                      ...   
56              4260                                0.000000   
                6240                           149352.494295   
                5820                                0.000000   
                4830                           135338.332211   
                6090                            13650.496790   

                             official_efig_hold_harmless  \
State FIPS Code District ID                                
1               190                        194698.670798   
                5                          592983.087034   
                30                         419186.018887   
                60                         168137.519722   
                90                         573132.739294   
...                                                  ...   
56              4260                            0.000000   
                6240                       148614.520996   
                5820                            0.

In [40]:
official_inputs = get_inputs(2021, use_official_children=True)
allocs1 = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'gaussian'})
 
allocs2 = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H'})

allocs3 = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_HB'})
 
allocs4 = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_no_noise'})

allocs4 = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H_prop'})

alloc5 = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H_reg'})
 

/Users/kentarohoffman/Desktop/dp-policy-master-2


/Users/kentarohoffman/Desktop/dp-policy-master-2/dp_policy/titlei/utils.py:70: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  combined.loc[:, "State FIPS Code"] = \


[INFO] Successfully imputed 2937 new indices
-- WARNING: dropping some balances from total budget --
Puerto Rico 412450771.2306171
County balances 723748.7996060263
Part D Subpart 2 104168000.98352575


In [26]:
allocs1 = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H_prop'})



In [10]:
allocs1['est_grant_total'].values[0:10]

array([ 586581.17223715, 1083796.59638323,  821909.24314364,
        476111.21378769, 1394885.35616709,  198722.89341764,
        671041.13487566,  330699.41409296,  537772.04052053,
       1302059.63592394])

In [11]:
allocs2['est_grant_total'].values[0:10]

array([      0.        ,       0.        ,       0.        ,
             0.        , 1014829.494237  ,  507414.7471185 ,
             0.        ,  522790.95157664,       0.        ,
             0.        ])

In [12]:
allocs3['est_grant_total'].values[0:10]

array([ 587318.38685238, 1085320.89650351,  824129.1236845 ,
        475972.97572488, 1398596.97155523,  198394.75270162,
        671653.96940346,  329975.18108606,  538375.18794802,
       1305589.16681728])

In [13]:
allocs4['est_grant_total'].values[0:10]

array([ 738117.08101063,  519623.22021811,  331942.59620402,
        141460.76884644,  554638.26201179,  147063.17553343,
        599457.51550769,  102243.92203753,  826354.98633069,
       1328702.68421718])

In [235]:
from tqdm import tqdm

#are the pop levers we are pulling matter?
n_sim = 10

df_allocation = np.empty([1,13190])
HB_allocation = np.empty([1,13190])
H_allocation = np.empty([1,13190])
H_prop_allocation = np.empty([1,13190])
H_reg_allocation = np.empty([1,13190])

df_allocations = np.empty([n_sim,13190])
HB_shrunk_allocations = np.empty([n_sim,13190])
H_shrunk_allocations = np.empty([n_sim,13190])
H_prop_shrunk_allocations = np.empty([n_sim,13190])
H_reg_shrunk_allocations = np.empty([n_sim,13190])

df_allocations_dp = np.empty([n_sim,13190])
HB_allocations_dp = np.empty([n_sim,13190])
H_prop_shrunk_allocations_dp = np.empty([n_sim,13190])

official_inputs = get_inputs(2021, use_official_children=True)

allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'gaussian_no_noise'})
df_allocation[0,:] = allocs['est_grant_total'].values

allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_HB'})
HB_allocation[0,:] = allocs['est_grant_total'].values

allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H'})
H_allocation[0,:] = allocs['est_grant_total'].values

allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H_prop'})
H_prop_allocation[0,:] = allocs['est_grant_total'].values

allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H_reg'})
H_reg_allocation[0,:] = allocs['est_grant_total'].values

#Sampling
for i in tqdm(range(0,n_sim)):
    allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'gaussian'})
    df_allocations[i,:] = allocs['est_grant_total'].values
    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_HB'})
    HB_shrunk_allocations[i,:] = allocs['est_grant_total'].values
    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H'})
    H_shrunk_allocations[i,:] = allocs['est_grant_total'].values
    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H_prop'})
    H_prop_shrunk_allocations[i,:] = allocs['est_grant_total'].values
    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H_reg'})
    H_reg_shrunk_allocations[i,:] = allocs['est_grant_total'].values  
    
#DP    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'gaussian_laplace'})
    df_allocations_dp[i,:] = allocs['est_grant_total'].values     
    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_HB_lapalce'})
    HB_allocations_dp[i,:] = allocs['est_grant_total'].values         
        
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H_prop_laplace'})
    H_prop_shrunk_allocations_dp[i,:] = allocs['est_grant_total'].values    

/Users/kentarohoffman/Desktop/dp-policy-master-2


/Users/kentarohoffman/Desktop/dp-policy-master-2/dp_policy/titlei/utils.py:70: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  combined.loc[:, "State FIPS Code"] = \


[INFO] Successfully imputed 2937 new indices
-- WARNING: dropping some balances from total budget --
Puerto Rico 412450771.2306171
County balances 723748.7996060263
Part D Subpart 2 104168000.98352575


100%|███████████████████████████████████████████| 10/10 [00:14<00:00,  1.48s/it]


In [241]:

for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H_prop_laplace'})
    H_prop_shrunk_allocations_dp[i,:] = allocs['est_grant_total'].values    

100%|███████████████████████████████████████████| 10/10 [00:15<00:00,  1.50s/it]


In [228]:
allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H_prop'})

In [229]:
allocs = funding(SonnenbergAuthorizer, official_inputs, Gaussian(epsilon = 1.0, delta = 0.0000001), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H_prop_laplace'})  

In [230]:
H_prop_shrunk_allocations.shape

(10, 13190)

In [251]:
from tqdm import tqdm

#are the pop levers we are pulling matter?
n_sim = 1000

df_allocation = np.empty([1,13190])
HB_allocation = np.empty([1,13190])
H_allocation = np.empty([1,13190])
H_prop_allocation = np.empty([1,13190])
H_reg_allocation = np.empty([1,13190])

df_allocations = np.empty([n_sim,13190])
HB_shrunk_allocations = np.empty([n_sim,13190])
H_shrunk_allocations = np.empty([n_sim,13190])
H_prop_shrunk_allocations = np.empty([n_sim,13190])
H_reg_shrunk_allocations = np.empty([n_sim,13190])

df_allocations_dp = np.empty([n_sim,13190])
HB_allocations_dp = np.empty([n_sim,13190])
H_prop_shrunk_allocations_dp = np.empty([n_sim,13190])

official_inputs = get_inputs(2021, use_official_children=True)

allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'gaussian_no_noise'})
df_allocation[0,:] = allocs['est_grant_total'].values

allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_HB'})
HB_allocation[0,:] = allocs['est_grant_total'].values

allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H'})
H_allocation[0,:] = allocs['est_grant_total'].values

allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H_prop'})
H_prop_allocation[0,:] = allocs['est_grant_total'].values

allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'JS_unequalvar_no_noise_H_reg'})
H_reg_allocation[0,:] = allocs['est_grant_total'].values

#Sampling
for i in tqdm(range(0,n_sim)):
    allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'gaussian'})
    df_allocations[i,:] = allocs['est_grant_total'].values
    df_allocations_dp[i,:] = allocs['dpest_grant_total'].values
    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_HB'})
    HB_shrunk_allocations[i,:] = allocs['est_grant_total'].values
    HB_allocations_dp[i,:] = allocs['dpest_grant_total'].values
    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H'})
    H_shrunk_allocations[i,:] = allocs['est_grant_total'].values
    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H_prop'})
    H_prop_shrunk_allocations[i,:] = allocs['est_grant_total'].values
    H_prop_shrunk_allocations_dp[i,:] = allocs['dpest_grant_total'].values
    
for i in tqdm(range(0,n_sim)): 
    allocs = funding(SonnenbergAuthorizer, official_inputs, Laplace(epsilon = 0.1, delta = 0.0), sppe,sampling_kwargs = {'distribution':'JSGaussian_unequalvar_H_reg'})
    H_reg_shrunk_allocations[i,:] = allocs['est_grant_total'].values  
    
    
    

/Users/kentarohoffman/Desktop/dp-policy-master-2


/Users/kentarohoffman/Desktop/dp-policy-master-2/dp_policy/titlei/utils.py:70: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  combined.loc[:, "State FIPS Code"] = \


[INFO] Successfully imputed 2937 new indices
-- WARNING: dropping some balances from total budget --
Puerto Rico 412450771.2306171
County balances 723748.7996060263
Part D Subpart 2 104168000.98352575


100%|███████████████████████████████████████| 1000/1000 [24:50<00:00,  1.49s/it]


In [248]:
allocs

Name  \
State FIPS Code District ID                                        
1               190               Alabaster City School District   
                5               Albertville City School District   
                30           Alexander City City School District   
                60                Andalusia City School District   
                90                 Anniston City School District   
...                                                          ...   
56              4260              Uinta County School District 6   
                6240           Washakie County School District 1   
                5820           Washakie County School District 2   
                4830             Weston County School District 1   
                6090             Weston County School District 7   

                             official_basic_hold_harmless  \
State FIPS Code District ID                                 
1               190                         451569.585546   
                5                           820486.906460   
                30                          548830.474149   
                60                          255888.455695   
                90                          690724.128654   
...                                                   ...   
56              4260                         40822.247350   
                6240                        223321.706091   
                5820                             0.000000   
                4830                        182499.458741   
                6090                         20411.123675   

                             official_concentration_hold_harmless  \
State FIPS Code District ID                                         
1               190                                  58092.627841   
                5                                   200836.024036   
                30                                  135670.119992   
                60                                   63255.265741   
                90                                  170746.031480   
...                                                           ...   
56              4260                                     0.000000   
                6240                                 24485.720477   
                5820                                  5762.953584   
                4830                                 93609.867864   
                6090                                     0.000000   

                             official_targeted_hold_harmless  \
State FIPS Code District ID                                    
1               190                            182516.637263   
                5                              553590.326339   
                30                             392958.114409   
                60                             157617.381627   
                90                             537272.595916   
...                                                      ...   
56              4260                                0.000000   
                6240                           149352.494295   
                5820                                0.000000   
                4830                           135338.332211   
                6090                            13650.496790   

                             official_efig_hold_harmless  \
State FIPS Code District ID                                
1               190                        194698.670798   
                5                          592983.087034   
                30                         419186.018887   
                60                         168137.519722   
                90                         573132.739294   
...                                                  ...   
56              4260                            0.000000   
                6240                       148614.520996   
                5820                            0.

In [250]:
np.savetxt("Gaussian/Gaussian_allocation.csv", df_allocation, delimiter=",")
np.savetxt("HB/HB_allocation.csv", HB_allocation, delimiter=",")
np.savetxt("H/H_allocation.csv", H_allocation, delimiter=",")
np.savetxt("H_prop/H_prop_allocation.csv", H_prop_allocation, delimiter=",")
np.savetxt("H_reg/H_reg_allocation.csv", H_reg_allocation, delimiter=",")

np.savetxt("Gaussian/Gaussian_allocations.csv", df_allocations, delimiter=",")
np.savetxt("HB/HB_shrunk_allocations.csv", HB_shrunk_allocations, delimiter=",")
np.savetxt("H/H_shrunk_allocations.csv", H_shrunk_allocations, delimiter=",")
np.savetxt("H_prop/H_prop_shrunk_allocations.csv", H_prop_shrunk_allocations, delimiter=",")
np.savetxt("H_reg/H_reg_allocations.csv", H_reg_shrunk_allocations, delimiter=",")

np.savetxt('Gaussian/Gaussian_allocations_dp.csv',df_allocations_dp,delimiter = ",")
np.savetxt('HB/HB_allocations_dp.csv',HB_allocations_dp,delimiter = ",")
np.savetxt('H_prop/H_prop_shrunk_allocations_dp.csv',H_prop_shrunk_allocations_dp,delimiter = ",")





